ADD API KEYS required below:

In [1]:

TAVILY_API_KEY = ""
COIN_API_KEY = ""
GOOGLE_GEMINI_API_KEY=""

# **Step 1 fetch latest news data from  Tavily**

In [2]:
import os
import requests

def fetch_tavily_data(query: dict) -> dict:
    """
    Fetches data from the Tavily API.

    Args:
        query (dict): A dictionary containing query parameters.

    Returns:
        dict: The response data from the Tavily API, or None if an error occurs.
    """
    query_params = query.get('params', {})

    print("param1", query_params)

    payload = {
        "api_key": TAVILY_API_KEY,
        "query": query_params.get('query'),
        "search_depth": query_params.get('search_depth', "basic"),
        "include_answer": True,  # query_params.get('include_answer', False),
        "max_results": query_params.get('max_results', 5),
        # "include_domains": query_params.get('include_domains', []),
        # "exclude_domains": query_params.get('exclude_domains', [])
    }

    try:
        response = requests.post("https://api.tavily.com/search", json=payload)
        response.raise_for_status()  # Raise an error for HTTP status codes 4xx/5xx
        print("Tavily response", response.json())
        return response.json()
    except requests.exceptions.RequestException as error:
        print("Error fetching data from Tavily:", error)
        return None


In [3]:
from datetime import datetime

# Get the current date in "YYYY-MM-DD" format
current_date = datetime.utcnow().strftime("%b %d, %Y")  # Example: "Jan 13, 2025"

query = {
    "params": {
        "query": f"BTC latest news {current_date}",
        "search_depth": "detailed",
        "max_results": 4,
    }
}
result = fetch_tavily_data(query)
print(query)

param1 {'query': 'BTC latest news 12 jan 2025', 'search_depth': 'detailed', 'max_results': 4}
Tavily response {'query': 'BTC latest news 12 jan 2025', 'follow_up_questions': None, 'answer': 'On January 12, 2025, the total cryptocurrency market cap was $3.36 trillion with a 1.01% increase on the day. Bitcoin analysts are predicting a recovery rally with a price range forecasted between $95,000 and $110,000 by the end of January. Additionally, there are expectations for Bitcoin to potentially double to $250,000 within a year and for the crypto market capitalization to surpass that of gold in the next decade. The current Bitcoin price prediction suggests a potential rise of 26.37% to reach $120,175 by February 11, 2025.', 'images': [], 'results': [{'title': 'TFUEL up +30.63%, BTC +0.21%, Cardano is The Coin of The Day - CoinCodex', 'url': 'https://coincodex.com/article/60861/daily-market-update-for-january-12-2025/', 'content': 'All prices and changes are presented at the time of publicat

In [4]:
result['results']

[{'title': 'TFUEL up +30.63%, BTC +0.21%, Cardano is The Coin of The Day - CoinCodex',
  'url': 'https://coincodex.com/article/60861/daily-market-update-for-january-12-2025/',
  'content': 'All prices and changes are presented at the time of publication: January 12, 2025, at 06:00 UTC; Market Overview. The total cryptocurrency market cap is currently $ 3.36T after a 1.01% increase on the day. The total crypto trading volume increased by 1.01% in the same time frame.',
  'score': 0.8306082,
  'raw_content': None},
 {'title': 'Bitcoin (BTC) price predictions for 2025',
  'url': 'https://www.cnbc.com/2024/12/31/bitcoin-btc-price-predictions-for-2025.html',
  'content': 'After a blistering 150% rally in bitcoin this year, crypto investors and industry executives told CNBC they’re expecting the flagship cryptocurrency to hit new all-time highs in 2025. Pension funds should also start including more bitcoin in their portfolio via U.S. spot ETFs next year thanks to anticipated reforms from th

# **Step 2: Fetch OPEN, High, LOW, Close data from CoinAPI for BTC/USDT**

In [5]:

from datetime import datetime, timedelta

import requests
import pandas as pd


# Function to fetch OHLCV data
def fetch_ohlcv(symbol, period_id, time_start):
    dt_object = datetime.strptime(time_start, "%Y-%m-%dT%H:%M:%S.%f")

    # Format to string without microseconds
    formatted_timestamp = dt_object.strftime("%Y-%m-%dT%H:%M:%S")
    print(formatted_timestamp)
    url = f"https://rest.coinapi.io/v1/ohlcv/{symbol}/history?period_id={period_id}&time_start={formatted_timestamp}"

    headers = {"X-CoinAPI-Key": COIN_API_KEY}  # Replace with your API key
    params = {
        "period_id": period_id,
        "time_start": time_start
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        if response.content:
            return response.json()
        else:
            print(f"Response for {period_id} is empty.")
            return []
    else:
        print(f"Failed to fetch {period_id} data. Status code: {response.status_code}")
        return []

# Function to determine overall sentiment
def determine_overall_sentiment(data):
    df = pd.DataFrame(data)
    df = df[['price_open', 'price_close']]

    # Calculate sentiment for each candle
    def sentiment(row):
        if row['price_close'] > row['price_open']:
            return "Bullish"
        elif row['price_close'] < row['price_open']:
            return "Bearish"
        else:
            return "Neutral"

    df['Sentiment'] = df.apply(sentiment, axis=1)

    # Calculate overall sentiment
    sentiment_counts = df['Sentiment'].value_counts(normalize=True)
    overall_sentiment = sentiment_counts.idxmax()  # Dominant sentiment
    return overall_sentiment, sentiment_counts.to_dict()

# Get current time and calculate start times for each timeframe
current_time = datetime.utcnow()
start_times = {
    "5MIN": (current_time - timedelta(days=3)).isoformat(),
    "1DAY": (current_time - timedelta(days=30)).isoformat(),
    "1MTH": (current_time - timedelta(days=90)).isoformat()
}

# Fetch data for different timeframes
symbol = "BINANCE_SPOT_BTC_USDT"  # Example symbol

data_5min = fetch_ohlcv(symbol, "5MIN", start_times["5MIN"])
data_1day = fetch_ohlcv(symbol, "1DAY", start_times["1DAY"])
data_1month = fetch_ohlcv(symbol, "1MTH", start_times["1MTH"])

# Determine overall sentiment for each timeframe
if data_5min:
    sentiment_5min, breakdown_5min = determine_overall_sentiment(data_5min)
    print("\n5-Minute Overall Sentiment:")
    print(f"Sentiment: {sentiment_5min}, Breakdown: {breakdown_5min}")

if data_1day:
    sentiment_1day, breakdown_1day = determine_overall_sentiment(data_1day)
    print("\n1-Day Overall Sentiment:")
    print(f"Sentiment: {sentiment_1day}, Breakdown: {breakdown_1day}")

if data_1month:
    sentiment_1month, breakdown_1month = determine_overall_sentiment(data_1month)
    print("\n1-Month Overall Sentiment:")
    print(f"Sentiment: {sentiment_1month}, Breakdown: {breakdown_1month}")


2025-01-10T12:13:40
2024-12-14T12:13:40
2024-10-15T12:13:40

5-Minute Overall Sentiment:
Sentiment: Bearish, Breakdown: {'Bearish': 0.53, 'Bullish': 0.47}

1-Day Overall Sentiment:
Sentiment: Bullish, Breakdown: {'Bullish': 0.5, 'Bearish': 0.5}

1-Month Overall Sentiment:
Sentiment: Bullish, Breakdown: {'Bullish': 0.5, 'Bearish': 0.5}


In [40]:
import plotly.graph_objects as go
import pandas as pd

# Function to plot 1-Day Candlestick Chart
def plot_candlestick(data):
    # Convert the fetched data into a DataFrame
    df = pd.DataFrame(data)

    # Convert 'time_period_start' to datetime
    df['time'] = pd.to_datetime(df['time_period_start'])

    # Plot using Plotly
    fig = go.Figure(data=[go.Candlestick(
        x=df['time'],
        open=df['price_open'],
        high=df['price_high'],
        low=df['price_low'],
        close=df['price_close'],
        name="Candlestick Chart"
    )])

    # Update layout to make the plot more readable
    fig.update_layout(
        title="1-Day BTC/USDT Candlestick Chart",
        xaxis_title="Date",
        yaxis_title="Price (USDT)",
        xaxis_rangeslider_visible=False,  # Disable the range slider
        template="plotly_dark"  # Optional: Change chart template (e.g., 'plotly_dark')
    )

    # Show the plot
    fig.show()

# Example to fetch and plot the data
symbol = "BINANCE_SPOT_BTC_USDT"
time_start = "2025-01-01T00:00:00"  # Use a proper timestamp in ISO format
period_id = "1DAY"

# Fetch the 1-day OHLCV data
data_1day = fetch_ohlcv(symbol, period_id, (current_time - timedelta(days=90)).isoformat())

# Plot the data if it's available
if data_1day:
    plot_candlestick(data_1day)


2024-10-15T12:14:24


In [ ]:
last_3_day_sentiment = sentiment_5min
last_30_day_sentiment = sentiment_1day
last_90_day_sentiment =sentiment_1month

In [ ]:

# Calculate RSI
def calculate_rsi(df, period=14):
    delta = df['price_close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Calculate Moving Averages
def calculate_moving_averages(df, short_window=9, long_window=21):
    df['MA_Short'] = df['price_close'].rolling(window=short_window).mean()
    df['MA_Long'] = df['price_close'].rolling(window=long_window).mean()
    return df

# Calculate ATR
def calculate_atr(df, period=14):
    df['High-Low'] = df['price_high'] - df['price_low']
    df['High-Close'] = abs(df['price_high'] - df['price_close'].shift())
    df['Low-Close'] = abs(df['price_low'] - df['price_close'].shift())
    df['True_Range'] = df[['High-Low', 'High-Close', 'Low-Close']].max(axis=1)
    df['ATR'] = df['True_Range'].rolling(window=period).mean()
    return df

# Fetch and analyze data
def analyze_symbol(symbol, period_id, time_start):
    data = fetch_ohlcv(symbol, period_id, time_start)
    if not data:
        print(f"No data fetched for {symbol} with period {period_id}.")
        return None

    df = pd.DataFrame(data)

    # Ensure necessary columns are present
    if set(['time_period_start', 'price_open', 'price_high', 'price_low', 'price_close']).issubset(df.columns):
        df['time_period_start'] = pd.to_datetime(df['time_period_start'])
        df = df.sort_values(by='time_period_start')

        # Perform analysis
        df = calculate_moving_averages(df)
        df['RSI'] = calculate_rsi(df)
        df = calculate_atr(df)

        print(f"\n{period_id} Analysis for {symbol}:")
        print(f"Latest RSI: {df['RSI'].iloc[-1]:.2f}")
        print(f"Latest ATR: {df['ATR'].iloc[-1]:.2f}")

        # Return the dataframe with analysis for further use
        return df
    else:
        print(f"Data for {symbol} is missing required columns.")
        return None

# Main execution
current_time = datetime.utcnow()
start_times = {
    "5MIN": (current_time - timedelta(days=3)).isoformat(),
    "1DAY": (current_time - timedelta(days=30)).isoformat(),
    "1MTH": (current_time - timedelta(days=90)).isoformat()
}

symbol = "BINANCE_SPOT_BTC_USDT"
analyze_data ={}
for period_id, time_start in start_times.items():
    analyze_data[period_id] = analyze_symbol(symbol, period_id, time_start)

# **Step 3: Initialize the Google Gemini LLM**

In [6]:
!pip install --quiet langchain
!pip install --quiet langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 2.2 MB/s eta 0:00:00


In [ ]:
pip install -U langchain-community

In [8]:
from langchain import PromptTemplate
from langchain.document_loaders import WebBaseLoader
from langchain.schema import StrOutputParser
from langchain.schema.prompt_template import format_document

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI


llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_GEMINI_API_KEY,
                 temperature=0.7, top_p=0.85)

# **Step 4: Generate the Prompt**

In [13]:

prompt = f'''
You are a highly skilled cryptocurrency analyst with expertise in technical analysis, sentiment evaluation, and market trends.

### Input Data
1. **Latest BTC News:**
{result['results']}

2. **Technical Chart Data:**
Analyze the following BTC/USDT price movements:

- 5-Minute Chart (last 3 days): Sentiment is {last_3_day_sentiment},
  Latest RSI: {analyze_data['5MIN']['RSI'].iloc[-1]}, Latest ATR: {analyze_data['5MIN']['ATR'].iloc[-1]}
- 1-Day Chart (last 30 days): Sentiment is {last_30_day_sentiment},
  Latest RSI: {analyze_data['1DAY']['RSI'].iloc[-1]}, Latest ATR: {analyze_data['1DAY']['ATR'].iloc[-1]}
- 1-Month Chart (last 90 days): Sentiment is {last_90_day_sentiment}

### Task
Analyze the provided data to suggest potential trades for BTC/USDT.

If there are valid trade opportunities, return a list of trades in JSON format. For each trade suggestion, include the following:
- `trade_type`: (buy/sell)
- `entry_price`: Suggested entry price
- `stop_loss`: Suggested stop loss
- `take_profit`: Suggested take profit
- `symbol`: BTC/USDT
- `reason`: A concise explanation of the trade rationale, including news impact, sentiment, and chart patterns.

If no valid trades are available, return an empty list (`[]`) along with a detailed reasoning for why no trades are recommended at this time.

### Rules:
- Avoid suggesting trades if there is high uncertainty or conflicting indicators.
- Base decisions on sentiment consistency, price patterns, and market momentum.
- Provide actionable and precise output.

### Output Format:
Output only in JSON format with the following structure:
{{
    "trades": [  // Array of trade objects if trades are available
        {{
            "trade_type": "buy",
            "entry_price": 42000,
            "stop_loss": 41500,
            "take_profit": 44000,
            "symbol": "BTC/USDT",
            "reason": "Bullish sentiment across all timeframes and positive market news indicating price breakout."
        }}
    ],
    "reasoning": "Detailed reasoning for the current market situation and why no trades are recommended."
}}

'''


In [31]:
ok = llm.invoke(prompt)


In [32]:
import json

# Assuming `ok.content` is the response in JSON format
formatted_content = json.loads(ok.content)  # Parse the JSON content from ok.content
print(json.dumps(formatted_content, indent=4))  # Print the formatted JSON

{
    "trades": [],
    "reasoning": "The technical analysis and news sentiment do not align to provide clear trading opportunities at this time. While the long-term sentiment is bullish, the 5-minute chart sentiment is bearish, indicating a potential correction. Given this mixed sentiment, it is prudent to wait for a more decisive trend or a clearer setup before initiating a trade."
}


# **Step 5: Take the trade on Hyperliquid**

In [ ]:
!pip install hyperliquid-python-sdk
from hyperliquid.info import Info
from hyperliquid.utils import constants
import json

info = Info(constants.TESTNET_API_URL, skip_ws=True)

# This is an example address, when you start trading this should be replaced with your address
user_state = info.user_state("0xcd5051944f780a621ee62e39e493c489668acf4d")
print(json.dumps(user_state, indent=2))

# if you're getting an error or no user_state here make sure your environment variables are set correctly and that all requirements are installed